In [1]:
import requests
import json
import datetime
import pandas as pd
import numpy as np

In [2]:
USERNAME ='OOIAPI-9N9UMLHV9W5GOP'
TOKEN= 'SJN6HXHH116OZ8'

In [6]:
refdes_streams = '/Users/knuth/Documents/ooi/repos/github/ooi_stats/sandbox/test.csv'

# need to specify a minute into day, else data from previous day is requested
begin_time_set = datetime.datetime(2016, 7, 8, 0,0,0)  

In [7]:
# wrangling

# request only parameter 7 (time)
parameter = '7' 

# read in csv (for now)
refdes_streams_df = pd.read_csv(refdes_streams, parse_dates=True) # read in csv (for now).

# prepare time stamp converters
ntp_epoch = datetime.datetime(1900, 1, 1)
unix_epoch = datetime.datetime(1970, 1, 1)
ntp_delta = (unix_epoch - ntp_epoch).total_seconds()
now = datetime.datetime.now()
# days = abs(begin_time_set.date() - now.date())
# days = int(days.days)
days = 5

# create some empty lists as inputs for your final data frame output
ref_des_list = []
stream_list = []
timestamp_list = []


In [8]:
%%time

# interate over reference designators, delivery methods and streams

for index, row in refdes_streams_df.iterrows():
    ref_des =  row['refdes']
    stream = row['stream']
    delivery_method = row['method']
    
    begin_time = begin_time_set
    begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    end_time = begin_time + datetime.timedelta(seconds=86400)
    end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')

    print ref_des,stream,delivery_method
    
#     while begin_time_set < now:
    for i in range(days):
        
        try:        
            response = requests.get('https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'+
                                    ref_des[:8]+'/'+ref_des[9:14]+'/'+ref_des[15:27]+'/'+
                                    delivery_method+'/'+stream+
                                    '?beginDT='+begin_time_str+'&endDT='+end_time_str+
                                    '&limit=1000&parameters='+parameter, 
                                    auth=(USERNAME, TOKEN))
            data = response.json()
            timestamp = data[0]['time']
            timestamp = datetime.datetime.utcfromtimestamp(timestamp - ntp_delta).replace(microsecond=0)
            timestamp = timestamp.date()
            
            print begin_time_str, end_time_str
            print timestamp

            ref_des_list.append(ref_des)
            stream_list.append(stream)
            timestamp_list.append(timestamp)

        except:
            data = response.json()
            # print "\n From", begin_time_str, end_time_str, ":"
            # print data['status']


        # move to next day
        begin_time = begin_time + datetime.timedelta(seconds=86400)
        begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
        end_time = end_time + datetime.timedelta(seconds=86400)
        end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')

CE01ISSM-MFD35-01-VEL3DD000 vel3d_cd_dcl_velocity_data_recovered recovered_host
CE01ISSM-MFD35-01-VEL3DD000 vel3d_cd_dcl_velocity_data telemetered
CE01ISSM-MFD35-02-PRESFA000 presf_abc_dcl_tide_measurement_recovered recovered_host
CE01ISSM-MFD35-02-PRESFA000 presf_abc_dcl_wave_burst_recovered recovered_host
CE01ISSM-MFD35-02-PRESFA000 presf_abc_tide_measurement_recovered recovered_inst
2016-07-08T00:00:00.000Z 2016-07-09T00:00:00.000Z
2016-07-08
2016-07-09T00:00:00.000Z 2016-07-10T00:00:00.000Z
2016-07-09
2016-07-10T00:00:00.000Z 2016-07-11T00:00:00.000Z
2016-07-10
2016-07-11T00:00:00.000Z 2016-07-12T00:00:00.000Z
2016-07-11
2016-07-12T00:00:00.000Z 2016-07-13T00:00:00.000Z
2016-07-12
CE05MOAS-GL319-03-ADCPAM000 adcp_bottom_track_earth recovered_host
CE05MOAS-GL319-03-ADCPAM000 adcp_velocity_glider recovered_host
CE05MOAS-GL319-04-DOSTAM000 dosta_abcdjm_glider_recovered recovered_host
CE05MOAS-GL319-04-DOSTAM000 dosta_abcdjm_glider_instrument telemetered
CE05MOAS-GL319-05-CTDGVM000 ctd

In [9]:
# convert lists to data frame
data_dict = {
    'refdes':ref_des_list,
    'stream':stream_list,
    'timestamp':timestamp_list}
ooi_data = pd.DataFrame(data_dict, columns = ['refdes', 'stream', 'timestamp'])

In [10]:
# collapse by unique ref_des and day with data
ooi_data = ooi_data.drop('stream', axis = 1)
ooi_data = ooi_data.drop_duplicates()
ooi_data = ooi_data[ooi_data.timestamp >= begin_time_set.date()]

In [11]:
ooi_data_days = ooi_data.groupby('refdes')['timestamp'].nunique().reset_index(name="days_of_data_count")
# add back any ref_des for which no data was found in the system
refdes_streams_df = refdes_streams_df.drop(['stream','method'], axis = 1)
ooi_data_days = pd.concat([ooi_data_days, refdes_streams_df])
ooi_data_days = ooi_data_days.fillna(value=0)
ooi_data_days = ooi_data_days.groupby('refdes')['days_of_data_count'].sum().reset_index(name='days_of_data_count')
ooi_data_days

,refdes,days_of_data_count
0,CE01ISSM-MFD35-01-VEL3DD000,0.0
1,CE01ISSM-MFD35-02-PRESFA000,5.0
2,CE05MOAS-GL319-03-ADCPAM000,0.0
3,CE05MOAS-GL319-04-DOSTAM000,0.0
4,CE05MOAS-GL319-05-CTDGVM000,0.0
5,CE05MOAS-GL320-01-PARADM000,5.0
6,CP05MOAS-A6264-03-CTDAVN000,0.0
7,CP05MOAS-A6264-04-NUTNRN000,0.0
8,CP05MOAS-A6264-05-ADCPAN000,0.0
9,CP05MOAS-A6264-06-PARADN000,0.0


In [12]:
ooi_data_days

,refdes,days_of_data_count
0,CE01ISSM-MFD35-01-VEL3DD000,0.0
1,CE01ISSM-MFD35-02-PRESFA000,5.0
2,CE05MOAS-GL319-03-ADCPAM000,0.0
3,CE05MOAS-GL319-04-DOSTAM000,0.0
4,CE05MOAS-GL319-05-CTDGVM000,0.0
5,CE05MOAS-GL320-01-PARADM000,5.0
6,CP05MOAS-A6264-03-CTDAVN000,0.0
7,CP05MOAS-A6264-04-NUTNRN000,0.0
8,CP05MOAS-A6264-05-ADCPAN000,0.0
9,CP05MOAS-A6264-06-PARADN000,0.0


In [13]:
# count ref_des on platform
ooi_data_days['platform'] = ooi_data_days.refdes.str[:8]
platform = ooi_data_days.groupby('platform')['refdes'].nunique().reset_index(name="inst_on_platform_count")
ooi_data_days = pd.merge(ooi_data_days,platform, on='platform')

# count ref_des on array
ooi_data_days['array'] = ooi_data_days.refdes.str[:2]
array = ooi_data_days.groupby('array')['refdes'].nunique().reset_index(name="inst_on_array_count")
ooi_data_days = pd.merge(ooi_data_days,array, on='array')
ooi_data_days

,refdes,days_of_data_count,platform,inst_on_platform_count,array,inst_on_array_count
0,CE01ISSM-MFD35-01-VEL3DD000,0.0,CE01ISSM,2,CE,6
1,CE01ISSM-MFD35-02-PRESFA000,5.0,CE01ISSM,2,CE,6
2,CE05MOAS-GL319-03-ADCPAM000,0.0,CE05MOAS,4,CE,6
3,CE05MOAS-GL319-04-DOSTAM000,0.0,CE05MOAS,4,CE,6
4,CE05MOAS-GL319-05-CTDGVM000,0.0,CE05MOAS,4,CE,6
5,CE05MOAS-GL320-01-PARADM000,5.0,CE05MOAS,4,CE,6
6,CP05MOAS-A6264-03-CTDAVN000,0.0,CP05MOAS,6,CP,6
7,CP05MOAS-A6264-04-NUTNRN000,0.0,CP05MOAS,6,CP,6
8,CP05MOAS-A6264-05-ADCPAN000,0.0,CP05MOAS,6,CP,6
9,CP05MOAS-A6264-06-PARADN000,0.0,CP05MOAS,6,CP,6


In [54]:
# for each ref_des - count days with data and divide by total days over time range requested
ooi_data_inst = pd.DataFrame(ooi_data_days, columns = ['refdes', 'days_of_data_count'])
ooi_data_inst['inst_percent'] = ooi_data_inst['days_of_data_count'] / days *100
ooi_data_inst.to_csv('output/instrument_percent.csv', columns = ['refdes', 'inst_percent'], index=False)

In [55]:
ooi_data_platform = pd.DataFrame(ooi_data_days, columns = ['platform', 'days_of_data_count','inst_on_platform_count'])
ooi_data_platform = ooi_data_platform.groupby(['platform','inst_on_platform_count'])['days_of_data_count'].sum().reset_index(name='days_of_data_count')
ooi_data_platform['platform_percent'] = ooi_data_platform['days_of_data_count'] / (days * ooi_data_platform['inst_on_platform_count']) *100
ooi_data_platform.to_csv('output/platform_percent.csv', columns = ['platform', 'platform_percent'], index=False)

In [56]:
ooi_data_array = pd.DataFrame(ooi_data_days, columns = ['array', 'days_of_data_count','inst_on_array_count'])
ooi_data_array = ooi_data_array.groupby(['array','inst_on_array_count'])['days_of_data_count'].sum().reset_index(name='days_of_data_count')
ooi_data_array['array_percent'] = ooi_data_array['days_of_data_count'] / (days * ooi_data_array['inst_on_array_count']) *100
ooi_data_array.to_csv('output/array_percent.csv', columns = ['array', 'array_percent'], index=False)


In [14]:
ooi_data_plotting = ooi_data
ooi_data_plotting['platform'] = ooi_data_plotting.refdes.str[:8]
ooi_data_plotting['array'] = ooi_data_plotting.refdes.str[:2]
ooi_data_plotting

,refdes,timestamp,platform,array
0,CE01ISSM-MFD35-02-PRESFA000,2016-07-08,CE01ISSM,CE
1,CE01ISSM-MFD35-02-PRESFA000,2016-07-09,CE01ISSM,CE
2,CE01ISSM-MFD35-02-PRESFA000,2016-07-10,CE01ISSM,CE
3,CE01ISSM-MFD35-02-PRESFA000,2016-07-11,CE01ISSM,CE
4,CE01ISSM-MFD35-02-PRESFA000,2016-07-12,CE01ISSM,CE
5,CE05MOAS-GL320-01-PARADM000,2016-07-08,CE05MOAS,CE
6,CE05MOAS-GL320-01-PARADM000,2016-07-09,CE05MOAS,CE
7,CE05MOAS-GL320-01-PARADM000,2016-07-10,CE05MOAS,CE
8,CE05MOAS-GL320-01-PARADM000,2016-07-11,CE05MOAS,CE
9,CE05MOAS-GL320-01-PARADM000,2016-07-12,CE05MOAS,CE


In [16]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [113]:
# plot instruments
yticks=[]

for index, row in ooi_data_plotting.iterrows():
    yticks.append(row["refdes"])
    
yticks = pd.unique(yticks)
yticks = yticks[::-1]
y = np.arange(len(yticks))
counter = -1

In [114]:
for x in y:
    print yticks[counter]
    for index, row in ooi_data.iterrows():
        if yticks[counter] == row["refdes"]:
            StartTime = row['timestamp']
            EndTime = row['timestamp'] + datetime.timedelta(seconds=86400)
            stream_time = np.array([StartTime,EndTime])
            stream_shape = np.full((stream_time.shape),y[counter])
            plt.plot(stream_time, stream_shape, linewidth=10, color='blue')
    counter = counter -1

RS03CCAL-MJ03F-05-BOTPTA301
RS01SBPS-PC01A-4A-CTDPFA103
CP03ISSM-SBD12-04-PCO2AA000
CP05MOAS-GL387-02-FLORTM000
CP05MOAS-GL387-03-CTDGVM000
CP05MOAS-GL387-04-DOSTAM000
CP05MOAS-GL387-05-PARADM000


In [115]:
plt.yticks(y, yticks)      
plt.xticks(rotation=20)
plt.tight_layout()
plt.grid()
plt.savefig('instruments.png', figsize=(16, 16), dpi=300,)
# plt.show()

In [116]:
# plot platforms
plt.close()
yticks=[]

for index, row in ooi_data_plotting.iterrows():
    yticks.append(row["platform"])
    
yticks = pd.unique(yticks)
yticks = yticks[::-1]
y = np.arange(len(yticks))
counter = -1

In [117]:
for x in y:
    print yticks[counter]
    for index, row in ooi_data.iterrows():
        if yticks[counter] == row["refdes"][:8]:
            StartTime = row['timestamp']
            EndTime = row['timestamp'] + datetime.timedelta(seconds=86400)
            stream_time = np.array([StartTime,EndTime])
            stream_shape = np.full((stream_time.shape),y[counter])
            plt.plot(stream_time, stream_shape, linewidth=10, color='blue')
    counter = counter -1

RS03CCAL
RS01SBPS
CP03ISSM
CP05MOAS


In [118]:
plt.yticks(y, yticks)      
plt.xticks(rotation=20)
plt.tight_layout()
plt.grid()
plt.savefig('platforms.png', figsize=(16, 16), dpi=300,)
# plt.show()

In [119]:
# plot arrays
plt.close()
yticks=[]

for index, row in ooi_data_plotting.iterrows():
    yticks.append(row["array"])
    
yticks = pd.unique(yticks)
yticks = yticks[::-1]
y = np.arange(len(yticks))
counter = -1

In [120]:
for x in y:
    print yticks[counter]
    for index, row in ooi_data.iterrows():
        if yticks[counter] == row["refdes"][:2]:
            StartTime = row['timestamp']
            EndTime = row['timestamp'] + datetime.timedelta(seconds=86400)
            stream_time = np.array([StartTime,EndTime])
            stream_shape = np.full((stream_time.shape),y[counter])
            plt.plot(stream_time, stream_shape, linewidth=10, color='blue')
    counter = counter -1

RS
CP


In [15]:
plt.yticks(y, yticks)      
plt.xticks(rotation=20)
plt.tight_layout()
plt.grid()
plt.savefig('arrays.png', figsize=(16, 16), dpi=300,)
# plt.show()

NameError: name 'plt' is not defined

In [32]:
# plot instruments
platforms = []

for index, row in ooi_data_plotting.iterrows():
    platforms.append(row["platform"])
    
platforms = pd.unique(platforms)



for i in platforms:
    yticks=[]
    
    for index, row in ooi_data_plotting.iterrows():
        if i == row["refdes"][:8]:
            yticks.append(row["refdes"])
    yticks = pd.unique(yticks)
    yticks = yticks[::-1]
    y = np.arange(len(yticks))
    counter = -1
    
    for x in y:
        for index, row in ooi_data.iterrows():
            if yticks[counter] == row["refdes"]:
                StartTime = row['timestamp']
                EndTime = row['timestamp'] + datetime.timedelta(seconds=86400)
                stream_time = np.array([StartTime,EndTime])
                stream_shape = np.full((stream_time.shape),y[counter])
                plt.plot(stream_time, stream_shape, linewidth=10, color='blue')
        counter = counter -1
    
    plt.yticks(y, yticks)      
    plt.xticks(rotation=20)
    plt.tight_layout()
    plt.grid()
    plt.savefig(str(i) + '.png', figsize=(16, 16), dpi=300,)
    plt.close()
    # plt.show()

In [28]:
platforms

array(['CE01ISSM', 'CE05MOAS', 'GA01SUMO', 'RS03AXPS'], dtype=object)

In [22]:
platforms

array(['CE01ISSM', 'CE05MOAS', 'GA01SUMO', 'RS03AXPS'], dtype=object)